In [50]:
import folium
import requests
import pandas as pd
import numpy
import re 

elec_table = pd.read_csv("/content/Home_Performance_Energy_Efficiency_Projects_Map.csv")

elec_table.head()


,Program:,Program link:,County:,Savings (electricity consumption reduction in kwh):,MEA contribution:,Total project cost:,Location 1
0,Home Performance Rebate Program,http://energy.maryland.gov/homeperformance/ind...,Harford,1.648,748.90,2354.00,"Abingdon, MD 210091716\n(39.4703499999527, -76..."
1,Home Performance Rebate Program,http://energy.maryland.gov/homeperformance/ind...,Montgomery,8.271,1829.70,11816.00,"Chevy Chase, MD 208153720\n(38.97726999991073,..."
2,Home Performance Rebate Program,http://energy.maryland.gov/homeperformance/ind...,Anne Arundel,1.464,796.81,2090.87,"Pasadena, MD 211221944\n(39.12812000013804, -7..."
3,Home Performance Rebate Program,http://energy.maryland.gov/homeperformance/ind...,Montgomery,1.436,817.98,2051.40,"Germantown, MD 208765632\n(39.19519999988023, ..."
4,Home Performance Rebate Program,http://energy.maryland.gov/homeperformance/ind...,Montgomery,2.528,745.45,3610.75,"Bethesda, MD 208161010\n(38.9582699996609, -77..."


In [51]:
#checking for missing values

#every row has a location value
elec_table[pd.notnull(elec_table["Location 1"])]["Location 1"].count()

#this data point has no longitude or latitude given in the location value, so it was removed
elec_table = elec_table.drop(elec_table[elec_table["Location 1"] == "Mount Airy, MD 211714782"].index)  

In [52]:
#extracting longitude and latitude out of "Location 1"
elec_table["longitude"] = elec_table["Location 1"].str.findall(r'([0-9]+.[0-9]+),')
elec_table["latitude"] = elec_table["Location 1"].str.findall(r', (-[0-9]+.[0-9]+)')

for index, crash in elec_table.iterrows():
  elec_table.at[index, 'longitude'] = float(re.search('\d{2}\.[0-9]+', crash["Location 1"]).group())
  elec_table.at[index, 'latitude'] = float(re.search('-\d{2}\.[0-9]+', crash["Location 1"]).group())

elec_table.columns

Index(['Program:', 'Program link:', 'County:',
       'Savings (electricity consumption reduction in kwh):',
       'MEA contribution:', 'Total project cost:', 'Location 1', 'longitude',
       'latitude'],
      dtype='object')

In [53]:
#creating map
map_osm = folium.Map(location=[39.0624, -77.1305], zoom_start=11)

#adding points to map
for index, crash in elec_table[elec_table["County:"] == "Montgomery"].iterrows():

  if crash['Savings (electricity consumption reduction in kwh):'] < 2.5:
    folium.Marker(location=[crash["longitude"], crash["latitude"]],
                    icon=folium.Icon(color='orange')).add_to(map_osm)
  elif crash['Savings (electricity consumption reduction in kwh):'] <= 5:
    folium.Marker(location=[crash["longitude"], crash["latitude"]],
                    icon=folium.Icon(color='red')).add_to(map_osm)
  else:
    folium.Marker(location=[crash["longitude"], crash["latitude"]],
                    icon=folium.Icon(color='purple')).add_to(map_osm)

    

map_osm

Key: 
Orange: < 2.5 kwh saved,
Red: < 2.5 and <= 5 kwh saved,
Purple: > 5 kwh saved

This dataset (available here: https://opendata.maryland.gov/Energy-and-Environment/Home-Performance-Energy-Efficiency-Projects-in-Mar/2tuk-rcgi/data) showcases different projects throught Maryland which recieved grants from the Maryland Energy Administration, in an attempt to reduce statewide energy consumption. All of the grants in this dataset came from the Home Performance Rebate Program. 

The map showcases these projects in Montgomery County, although the dataset has data for all of Maryland. It is organized by how many kwh were saved by each project, with orange being less than 2.5 kwh, red being between 2.5 and 5 kwh, and purple being above 5 kwh saved. Using this dataset, I thought color coding the points in this way was the most effective way to differentiate the data. 

